# Image Text Extraction and Information Retrieval

## The Question

Being able to take pictures of text, recognize, convert into computer text, and search either typed or hand-written is a very interesting and useful concept.
**How can we use python image processing and information retrieval libraries to extract text from images and search through the results with specific queries?**

## The Data

Our data consists of **typed and hand-written document images and scans**. We created a dataset that is combined from multiple other datasets. It consists of approximately **1509 data points.**

## The Approach

Since all these images have different document styles and structures, we want to see how well the image text detection library we're using will be able to convert the document into computer text thus allowing us to search through the documents. We will not be implementing the document image word extraction or the search engine from scratch, but we will be using multiple libraries in order to see how these machine learning and natural language processing concepts and libraries work.

## Our Hypothesis

Since some characters share similar features with other characters, and handwritten characters are less consistent than typed characters, we expect a higher accuracy rate when we convert images with typed text to regular text than when we convert images with handwritten text to regular text.

## Why Bother?

Teaching a machine to recognize physical typed and hand-written documents from pictures and scans as well as being able to search through these documents is a very important and useful machine learning concept.

There are many applications such as being able to scan these documents into a database and being able to search for information within the documents.

## Basic Setup

### PIL, Pytesseract

We first loaded in the machine learning libraries. **PIL** allows us to process the images. **Pytesseract** allows us to convert the text in the image into a text string. **Pandas** allows us to manipulate data frames easily. **Elasticsearch** allows us to initialize a search engine for our data.

In [ ]:
!pip install pytesseract
!pip install pytesseract
!pip install pandas
!pip install elasticsearch

In [ ]:
# adds image processing capabilities
from PIL import Image

# will convert the image text into a text string
import pytesseract

# allows us to open files
import os

# allows us to manipulate data frames easily
import pandas as pd

# allows us to initialize a search engine for our data
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# allows us to convert a dataframe to csv format
import csv

# provides access to some ariables and functions from the interpreter
import sys

## The Data

## Convert Documents

We start off by creating an empty list named *images* for storing the source path of each image and an empty list named *results* for storing the results of converting each image to text.The image data is stored in two separate folders, one for images with typed text and one for images with handwritten text. For each image in these two folders, we append the pathname to *images*, open the image using Image class from the PIL module, convert the image text to string text, and append the extracted string text to *results*.
#### Only run if you intend to create a new datastore since it takes about an hour to run.

In [ ]:
#list for storing each image and it's extracted text
images = []
results = []

for filename in os.listdir('Data/Typed'):
    if filename.endswith('.png'):
        with open(os.path.join('Data/Typed', filename)) as f:
            images.append('Data/Typed/' + filename)
            # assigning an image from the source path
            img = Image.open('Data/Typed/' + filename)
            # converts the image to result and saves it into result variable
            result = pytesseract.image_to_string(img)
            results.append(result)

for filename in os.listdir('Data/Written'):
    if filename.endswith('.jpg'):
        with open(os.path.join('Data/Written', filename)) as f:
            images.append('Data/Written/' + filename)
            # assigning an image from the source path
            img = Image.open('Data/Written/' + filename)
            # converts the image to result and saves it into result variable
            result = pytesseract.image_to_string(img)
            results.append(result)
    if filename.endswith('.png'):
        with open(os.path.join('Data/Written', filename)) as f:
            images.append('Data/Written/' + filename)
            # assigning an image from the source path
            img = Image.open('Data/Written/' + filename)
            # converts the image to result and saves it into result variable
            result = pytesseract.image_to_string(img)
            results.append(result)

## Store in Datastore

Here we create and convert a dictionary that stores the images and results into a Pandas DataFrame. Then we export the DataFrame to a csv file.

#### Only run after running **Convert Documents**.

In [ ]:
df = pd.DataFrame({
        'image': images,
        'result': results
})

df.to_csv('output.csv', index=False)

## Elasticsearch

Download Elasticsearch to your computer here: https://www.elastic.co/guide/en/elasticsearch/reference/current/install-elasticsearch.html

Run Elasticsearch by following these steps: https://www.elastic.co/guide/en/elasticsearch/reference/6.8/getting-started-install.html

Here we are creating an Elasticsearch index from the csv created from the data. The data consists of the image file path and the contents of the converted document.

In [ ]:
with open('output.csv') as f:
    reader = csv.DictReader(f)
    es.indices.delete(index='documents')
    helpers.bulk(es, reader, index='documents')
    

Here we get inputted searches from user input which then employs es.search in order to search through creaated index for searches inputted by the user. Once the user exits it outputs the search results.

In [ ]:
searching = True

while searching:
    search = input("Search (to exit type 'exit'): ")

    if (search == 'exit') or (search == 'Exit'):
        break
    else:
        print('Search: ' + search)
        print()
        result = es.search(
            index="documents",
            body={
                "query": {
                    "multi_match": {
                        "query": search,
                        "fields": ["result"]
                    }
                }
            }
        )
    all_hits = result['hits']['hits']

    for hit in all_hits:
        print("Image:", hit['_source']['image'])
        im = Image.open(hit['_source']['image'])
        im.show()
        print("Result:", hit['_source']['result'])


## Voila!